<a href="https://colab.research.google.com/github/LyesDemri/MLP-for-MNIST/blob/main/My_NN_vs_Tensorflow_for_Deep_Learning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Load data from files

In [1]:
%reset -f

import numpy as np
import matplotlib.pyplot as plt
import csv

#load train file
file = open('sample_data/mnist_train_small.csv','r');
csvreader = csv.reader(file);
X = [0]*20000;
label = [0]*20000;
k=0;
for row in csvreader:
  label[k] = row[0];
  X[k] = row[1:];
  k+=1;
X = np.double(np.transpose(np.array(X)))/255;

T = np.zeros((10,20000));

for i in range(20000):
  T[int(label[i]),i] = 1;

#load test file
file = open('sample_data/mnist_test.csv','r');
csvreader = csv.reader(file);
X_test = [0]*10000;
label_test = [0]*10000;
k=0;
for row in csvreader:
  label_test[k] = row[0];
  X_test[k] = row[1:];
  k+=1;

X_test = np.double(np.transpose(np.array(X_test)))/255;

T_test = np.zeros((10,10000));

for i in range(10000):
  T_test[int(label_test[i]),i] = 1;

print('Data has been loaded')

Data has been loaded


Recognize digits with my own code first:

In [3]:
#Generate network:
#layers = ['ReLU']*9 + ['sigmoid'];
layers = ['sigmoid','sigmoid']
#layers = ['sigmoid']*10;
print(layers)
n_layers = len(layers);
ILU = 784;  #input layer units
HLU = 10;   #hidden layer units
OLU = 10;   #output layer units
W = [0]*n_layers;
b = [0]*n_layers;

W[0] = np.random.rand(HLU,ILU)*0.1-0.05;
b[0] = np.random.rand(HLU,1)*0.1-0.05;
for i in range(1,n_layers-1):
  W[i] = np.random.rand(HLU,HLU)*0.1-0.05;
  b[i] = np.random.rand(HLU,1)*0.1-0.05;
W[n_layers-1] = np.random.rand(OLU,HLU)*0.1-0.05;
b[n_layers-1] = np.random.rand(OLU,1)*0.1-0.05;

#start training:

def logsig(x):
  y = np.zeros(np.shape(x));
  for i in range(len(x)):
    y[i] = 1/(1+np.exp(-x[i]));
  return y;

def ReLU(x):
  y = np.zeros(np.shape(x));
  for i in range(len(x)):
    y[i] = np.max([0,x[i]]);
  return y;
def derive(Y,layer_type):
  derivative = np.zeros(np.shape(Y));
  if layer_type == 'sigmoid':
    for i in range(len(Y)):
      derivative[i] = Y[i]*(1-Y[i]);
  elif layer_type == 'ReLU':
    for i in range(len(Y)):
      derivative[i] = 1;
  return derivative;

epochs = 10;
errors_list = np.zeros(epochs);
for epoch in range(epochs):
  for i in range(20000):
    #forward:
    Y = [0]*(n_layers+1);
    Y[0] = X[:,[i]];
    for l in range(n_layers):
      if layers[l] == 'sigmoid':
        Y[l+1] = logsig(W[l]@Y[l] + b[l]);
      elif layers[l] == 'ReLU':
        Y[l+1] = ReLU(W[l]@Y[l] + b[l]);

    dEdb = [0]*n_layers;
    dEdW = [0]*n_layers;
    d = 2*(Y[-1]-T[:,[i]])/10;
    for l in range(n_layers-1,-1,-1):
      d = d*derive(Y[l+1],layers[l]);
      dEdb[l] = d;
      dEdW[l] = d@Y[l].transpose();
      d = W[l].transpose()@d;
    
    for l in range(n_layers):
      W[l] = W[l] - 0.1*dEdW[l];
      b[l] = b[l] - 0.1*dEdb[l];
    
  errors = 0;
  for i in range(20000):
    #forward:
    Y = [0]*(n_layers+1);
    Y[0] = X[:,[i]];
    for l in range(n_layers):
      if layers[l] == 'sigmoid':
        Y[l+1] = logsig(W[l]@Y[l] + b[l]);
      elif layers[l] == 'ReLU':
        Y[l+1] = ReLU(W[l]@Y[l] + b[l]);
    a = np.argmax(Y[n_layers]);
    ca = np.argmax(T[:,[i]]);
    if ca!=a:
      errors+=1;

  errors_list[epoch] = errors*100/20000;
  print("Epoch ",epoch+1, ": ", 100-errors*100/20000, "% accuracy")

errors = 0;
for i in range(10000):
  #forward:
  Y = [0]*(n_layers+1);
  Y[0] = X_test[:,[i]];
  for l in range(n_layers):
    if layers[l] == 'sigmoid':
      Y[l+1] = logsig(W[l]@Y[l] + b[l]);
    elif layers[l] == 'ReLU':
      Y[l+1] = ReLU(W[l]@Y[l] + b[l]);
  a = np.argmax(Y[n_layers]);
  ca = np.argmax(T_test[:,[i]]);
  if ca!=a:
    errors+=1;
errors_on_test = errors *100/10000;
print("Accuracy on test set: ",100-errors_on_test);

['sigmoid', 'sigmoid']
Epoch  1 :  30.22 % accuracy
Epoch  2 :  64.545 % accuracy
Epoch  3 :  84.75 % accuracy
Epoch  4 :  88.615 % accuracy
Epoch  5 :  89.57 % accuracy
Epoch  6 :  90.245 % accuracy
Epoch  7 :  90.615 % accuracy
Epoch  8 :  91.02 % accuracy
Epoch  9 :  91.37 % accuracy
Epoch  10 :  91.645 % accuracy
Accuracy on test set:  91.08


Now let's use Tensorflow

In [5]:
import tensorflow as tf

model = tf.keras.models.Sequential([tf.keras.layers.Input((784,)),
    tf.keras.layers.Dense(units = 10, activation = 'sigmoid', kernel_initializer = 'RandomUniform', bias_initializer = 'RandomUniform'),  
    tf.keras.layers.Dense(units = 10, activation = 'sigmoid', kernel_initializer = 'RandomUniform', bias_initializer = 'RandomUniform')]);

model.compile(optimizer = tf.keras.optimizers.SGD(0.1), loss = 'MSE', metrics = 'accuracy')
model.fit(X.transpose(),T.transpose(),epochs = 10, batch_size = 1);
model.evaluate(X_test.transpose(), T_test.transpose(), batch_size = 1)

Epoch 1/10
20000/20000 [==============================] - 34s 2ms/step - loss: 0.0893 - accuracy: 0.1760
Epoch 2/10
20000/20000 [==============================] - 33s 2ms/step - loss: 0.0653 - accuracy: 0.4817
Epoch 3/10
20000/20000 [==============================] - 31s 2ms/step - loss: 0.0434 - accuracy: 0.7465
Epoch 4/10
20000/20000 [==============================] - 33s 2ms/step - loss: 0.0297 - accuracy: 0.8637
Epoch 5/10
20000/20000 [==============================] - 30s 2ms/step - loss: 0.0230 - accuracy: 0.8918
Epoch 6/10
20000/20000 [==============================] - 30s 1ms/step - loss: 0.0198 - accuracy: 0.9015
Epoch 7/10
20000/20000 [==============================] - 32s 2ms/step - loss: 0.0179 - accuracy: 0.9071
Epoch 8/10
20000/20000 [==============================] - 30s 1ms/step - loss: 0.0167 - accuracy: 0.9117
Epoch 9/10
20000/20000 [==============================] - 30s 2ms/step - loss: 0.0159 - accuracy: 0.9153
Epoch 10/10
10000/10000 [==============================

[0.01544282864779234, 0.9136000275611877]